# Simulation Testing

Testing the basic functionality of simpy

## Basic operation of a car

In [35]:
import simpy
import simpy.rt as srt
import random
from datetime import datetime, timedelta

In [8]:
data = []

def test_process(env, data):
    val = 0
    for i in range(5):
        val += env.now
        data.append(val)
        yield env.timeout(1)

In [9]:
env = simpy.Environment()

p = env.process(test_process(env, data))

env.run(p)

In [10]:
print(data)

[0, 1, 3, 6, 10]


# More complex routine with a car

A car will alternate between parking and driving

In [12]:
def car(env):
    while True:
        print('start parking at: %d' % env.now)
        parking_duration = 10 + random.randint(-5, 20)
        yield env.timeout(parking_duration)

        print('start driving at: %d' % env.now)
        trip_duration = 20 + random.randint(-10, 60)
        yield env.timeout(trip_duration)

        

In [29]:
env = simpy.Environment()
env.process(car(env))
env.run(until=60*3)

start parking at: 0
start driving at: 5
start parking at: 53
start driving at: 64
start parking at: 99
start driving at: 128


In [27]:
env = srt.RealtimeEnvironment(factor=0.05)
env.process(car(env))
env.run(until=60*3)

start parking at: 0
start driving at: 21
start parking at: 34
start driving at: 52
start parking at: 67
start driving at: 76
start parking at: 149
start driving at: 168


## Single station test

Test with 5 cars, each takes in average 120 mins to charge, with 3 spots available at charging station, simulate until all cars finished charging.

In [52]:
def car(env, name, bcs, driving_time, charge_duration, now_time):
    # Simulate driving to the BCS
    yield env.timeout(driving_time)

    # Request one of its charging spots
    print('%s arriving at %s' % (name, now_time + timedelta(minutes=env.now)))
    with bcs.request() as req:
        yield req

        # Charge the battery
        print('%s starting to charge at %s' % (name, now_time + timedelta(minutes=env.now)))
        yield env.timeout(charge_duration)
        print('%s leaving the charging station at %s' % (name, now_time + timedelta(minutes=env.now)))

In [59]:
env = simpy.Environment()
# env = srt.RealtimeEnvironment(factor=1)
bcs = simpy.Resource(env, capacity=3)
now = datetime.now()
for i in range(5):
    env.process(car(env, 'Car %d' % i, bcs, 20 + random.randint(-10, 60), 120 + random.randint(-10, 10), now))

In [60]:
env.run()

Car 4 arriving at 2024-03-16 23:15:17.994967
Car 4 starting to charge at 2024-03-16 23:15:17.994967
Car 3 arriving at 2024-03-16 23:28:17.994967
Car 3 starting to charge at 2024-03-16 23:28:17.994967
Car 0 arriving at 2024-03-16 23:39:17.994967
Car 0 starting to charge at 2024-03-16 23:39:17.994967
Car 2 arriving at 2024-03-16 23:54:17.994967
Car 1 arriving at 2024-03-17 00:10:17.994967
Car 4 leaving the charging station at 2024-03-17 01:11:17.994967
Car 2 starting to charge at 2024-03-17 01:11:17.994967
Car 0 leaving the charging station at 2024-03-17 01:36:17.994967
Car 1 starting to charge at 2024-03-17 01:36:17.994967
Car 3 leaving the charging station at 2024-03-17 01:38:17.994967
Car 2 leaving the charging station at 2024-03-17 03:06:17.994967
Car 1 leaving the charging station at 2024-03-17 03:40:17.994967


## Learning points

Have to look at how realtime simulation works in simpy

Alternatively, look at how we can run a routine in parallel